In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, floor, col, array, element_at,when, lit

In [41]:
from pyspark import SparkContext, SQLContext

In [42]:
spark = SparkSession.builder.master("local[*]").appName("Exercicio Intro").getOrCreate()

In [43]:
df_nomes = spark.read.csv("nomes_aleatorios.txt")

In [44]:
df_nomes.show(5)

+------------------+
|               _c0|
+------------------+
|      Jess Nichols|
| Luther Goicoechea|
|     Mary Williams|
|    Jeffrey Aleman|
|Jacquelyne Bedford|
+------------------+
only showing top 5 rows



In [45]:
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")

df_nomes.show(10)

+------------------+
|             Nomes|
+------------------+
|      Jess Nichols|
| Luther Goicoechea|
|     Mary Williams|
|    Jeffrey Aleman|
|Jacquelyne Bedford|
|   Thomas Routzahn|
|     Santos Vieyra|
|      Betty Probst|
|      Janet Pearce|
|      Andrew Spies|
+------------------+
only showing top 10 rows



In [46]:
df_nomes = df_nomes.withColumn(
    "Escolaridade",
    when(rand() < 1/3, "Fundamental")
    .when(rand() < 0.5, "Medio")
    .otherwise("Superior")
)

In [47]:
df_nomes.show(5)

+------------------+------------+
|             Nomes|Escolaridade|
+------------------+------------+
|      Jess Nichols|       Medio|
| Luther Goicoechea|       Medio|
|     Mary Williams| Fundamental|
|    Jeffrey Aleman|    Superior|
|Jacquelyne Bedford|       Medio|
+------------------+------------+
only showing top 5 rows



In [48]:
paises = ["Uruguai", "Venezuela", "Guiana Francesa","Argentina", "Bolívia", "Brasil", "Chile", "Colômbia","Equador", "Guiana", "Paraguai", "Peru", "Suriname"]

In [49]:
# Criar uma coluna com índice aleatório de 1 a 13
df_nomes = df_nomes.withColumn("indice_pais", floor(rand() * 13) + 1)

# Adicionar a coluna "Pais" usando o índice aleatório para buscar no array de países
df_nomes = df_nomes.withColumn("Pais", element_at(array(*[lit(p) for p in paises]), col("indice_pais").cast("int")))

# (Opcional) Remover a coluna auxiliar
df_nomes = df_nomes.drop("indice_pais")

In [50]:
df_nomes.show(5)

+------------------+------------+---------+
|             Nomes|Escolaridade|     Pais|
+------------------+------------+---------+
|      Jess Nichols|       Medio|   Brasil|
| Luther Goicoechea|       Medio|   Guiana|
|     Mary Williams| Fundamental| Paraguai|
|    Jeffrey Aleman|    Superior|Venezuela|
|Jacquelyne Bedford|       Medio| Colômbia|
+------------------+------------+---------+
only showing top 5 rows



In [51]:
df_nomes = df_nomes.withColumn("AnoNascimento", floor(rand() * 66) + 1945)

In [52]:
df_nomes.show(5)

+------------------+------------+---------+-------------+
|             Nomes|Escolaridade|     Pais|AnoNascimento|
+------------------+------------+---------+-------------+
|      Jess Nichols|       Medio|   Brasil|         2006|
| Luther Goicoechea|       Medio|   Guiana|         1981|
|     Mary Williams| Fundamental| Paraguai|         1953|
|    Jeffrey Aleman|    Superior|Venezuela|         2005|
|Jacquelyne Bedford|       Medio| Colômbia|         2009|
+------------------+------------+---------+-------------+
only showing top 5 rows



In [53]:
df_select = (
    df_nomes               # DataFrame original
      .select("*")         # mantém todas as colunas
      .where(col("AnoNascimento") >= 2001)
)

# Mostra 10 nomes do novo DataFrame
df_select.select("Nomes").show(10, truncate=False)

+------------------+
|Nomes             |
+------------------+
|Jess Nichols      |
|Jeffrey Aleman    |
|Jacquelyne Bedford|
|Andrew Spies      |
|Shirley Cozart    |
|Devon Bronk       |
|Raymond Whitaker  |
|Raymond Jenkins   |
|Donald Brumit     |
|Mary Boyce        |
+------------------+
only showing top 10 rows



In [54]:
# 1) Registrar o DataFrame como visão temporária
df_nomes.createOrReplaceTempView("pessoas")

# 2) Consultar via SQL as pessoas nascidas no século XXI
df_select = spark.sql("""
    SELECT *
    FROM   pessoas
    WHERE  AnoNascimento >= 2001   -- ajuste para >= 2000 se preferir
""")

# 3) Exibir 10 nomes para conferir
df_select.select("Nomes").show(10, truncate=False)

+------------------+
|Nomes             |
+------------------+
|Jess Nichols      |
|Jeffrey Aleman    |
|Jacquelyne Bedford|
|Andrew Spies      |
|Shirley Cozart    |
|Devon Bronk       |
|Raymond Whitaker  |
|Raymond Jenkins   |
|Donald Brumit     |
|Mary Boyce        |
+------------------+
only showing top 10 rows



In [55]:
# Millennials: 1980 ≤ AnoNascimento ≤ 1994
millennials_count = (
    df_nomes
      .filter((col("AnoNascimento") >= 1980) & (col("AnoNascimento") <= 1994))
      .count()
)

print(f"Total de pessoas da geração Millennials: {millennials_count}")

Total de pessoas da geração Millennials: 2270855


In [56]:
# registrar visão temporária se ainda não existir
df_nomes.createOrReplaceTempView("pessoas")

millennials_sql = spark.sql("""
    SELECT COUNT(*) AS qtd_millennials
    FROM   pessoas
    WHERE  AnoNascimento BETWEEN 1980 AND 1994
""")

millennials_sql.show()

+---------------+
|qtd_millennials|
+---------------+
|        2270855|
+---------------+



In [57]:
geracoes_por_pais = spark.sql("""
    SELECT
        Pais,
        CASE
            WHEN AnoNascimento BETWEEN 1944 AND 1964 THEN 'Baby Boomers'
            WHEN AnoNascimento BETWEEN 1965 AND 1979 THEN 'Geração X'
            WHEN AnoNascimento BETWEEN 1980 AND 1994 THEN 'Millennials'
            WHEN AnoNascimento BETWEEN 1995 AND 2015 THEN 'Geração Z'
        END AS Geracao,
        COUNT(*) AS Quantidade
    FROM   pessoas
    GROUP BY Pais, Geracao
    ORDER BY Pais, Geracao, Quantidade
""")

# exibe o resultado e deixa em DataFrame para uso posterior
geracoes_por_pais.show(truncate=False)

+---------+------------+----------+
|Pais     |Geracao     |Quantidade|
+---------+------------+----------+
|Argentina|Baby Boomers|233482    |
|Argentina|Geração X   |175457    |
|Argentina|Geração Z   |185855    |
|Argentina|Millennials |174660    |
|Bolívia  |Baby Boomers|233655    |
|Bolívia  |Geração X   |175493    |
|Bolívia  |Geração Z   |186506    |
|Bolívia  |Millennials |175205    |
|Brasil   |Baby Boomers|233242    |
|Brasil   |Geração X   |174765    |
|Brasil   |Geração Z   |187099    |
|Brasil   |Millennials |174954    |
|Chile    |Baby Boomers|233028    |
|Chile    |Geração X   |175313    |
|Chile    |Geração Z   |186271    |
|Chile    |Millennials |174411    |
|Colômbia |Baby Boomers|233189    |
|Colômbia |Geração X   |175465    |
|Colômbia |Geração Z   |186425    |
|Colômbia |Millennials |174388    |
+---------+------------+----------+
only showing top 20 rows

